In [ ]:
#install dependencies
%pip install requests pandas 
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118


In [14]:
#get data 
%cd F1dataset 
circuits = pd.read_csv('circuits.csv')
drivers = pd.read_csv('drivers.csv')
lap_times = pd.read_csv('lap_times.csv')
pit_stops = pd.read_csv('pit_stops.csv') 
qualifying = pd.read_csv('qualifying.csv')
results = pd.read_csv('results.csv')
seasons = pd.read_csv('seasons.csv')
status = pd.read_csv('status.csv')

#Explore the datasets
datasets = {
    "Circuits": circuits,
    "Drivers": drivers,
    "Lap Times": lap_times,
    "Pit Stops": pit_stops,
    "Qualifying": qualifying,
    "Results": results,
    "Seasons": seasons,
    "Status": status
}

[Errno 2] No such file or directory: 'F1dataset'
/home/penguin/Documents/Code/F1RaceOutcomePrediction/F1 dataset


NameError: name 'pd' is not defined

In [200]:
#Data
lap_times = datasets['Lap Times']
drivers = datasets['Drivers']
circuits = datasets['Circuits']


#Data merging
merged_data = lap_times.merge(drivers, on='driverId', how='inner')
merged_data = merged_data.merge(circuits, left_on='driverId', right_on='circuitId', how='inner')

#Convert lap times to seconds
import pandas as pd

def convert_to_seconds(time_str):
    if isinstance(time_str, str):
        minutes, seconds = time_str.split(':')
        return int(minutes) * 60 + float(seconds)
    return time_str

# Ensure the correct column name is used
if 'milliseconds' in merged_data.columns:
    merged_data['lapTime'] = merged_data['milliseconds'].apply(lambda x: x / 1000)
else:
    raise KeyError("Column 'milliseconds' not found in the merged dataframe")


In [ ]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
import numpy as np

# Ensure all lap times are converted to seconds before transforming
merged_data['lapTime'] = merged_data['lapTime'].apply(convert_to_seconds)
merged_data = merged_data.dropna(subset=['lapTime'])
feature_columns = ['driverId', 'circuitId', 'lapTime']  

# Separate features and target variable
X = merged_data[feature_columns]
y = merged_data['lapTime']  # If you're predicting future lap times, adjust accordingly

# Define categorical and numerical features
categorical_features = ['driverId', 'circuitId']
numerical_features = ['lapTime']  # Include other numerical features if any

# Transformer and scaler
one_hot = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

transformer = ColumnTransformer(
    transformers=[
        ('one_hot', one_hot, categorical_features),
        ('scaler', StandardScaler(), numerical_features)
    ],
    remainder='drop'  # Exclude other columns not specified
)

# Apply the transformer to features
X_transformed = transformer.fit_transform(X)
print("Transformed features shape:", X_transformed.shape)

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X_transformed, y, test_size=0.2, random_state=42
)
print("Training and testing data prepared successfully.")


In [207]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Convert the data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)

#define neural network 
class LapTimePredictor(nn.Module):
    def __init__(self, input_size):
        super(LapTimePredictor, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [211]:
#initialize the model, loss function, and optimizer
input_size = X_train_tensor.shape[1]
model = LapTimePredictor(input_size)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

#Prepare the data loaders
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [ ]:
#Training loop 
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    for X_batch, y_batch in train_loader:
        #forward pass
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)

        #backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item():.4f}")

# Evaluate the model on the test set
model.eval()
with torch.no_grad():
    test_predictions = model(X_test_tensor)
    test_loss = criterion(test_predictions, y_test_tensor)
    print(f'Test Loss: {test_loss.item():.4f}')

In [ ]:
import matplotlib.pyplot as plt

# Convert predictions and true values to numpy for easy plotting
predicted = test_predictions.numpy().flatten()
actual = y_test_tensor.numpy().flatten()

# Plotting the results
plt.figure(figsize=(10, 6))
plt.scatter(actual, predicted, alpha=0.6)
plt.plot([min(actual), max(actual)], [min(actual), max(actual)], color='red', linestyle='--')  # Diagonal line
plt.xlabel('Actual Lap Times')
plt.ylabel('Predicted Lap Times')
plt.title('Actual vs. Predicted Lap Times')
plt.show()


In [1]:
def predict_average_race_lap_times(circuit_id, average_lap_time):
    # Get unique driver IDs from the dataset
    driver_ids = merged_data['driverId'].unique()
    
    # List to store predicted lap times for each driver
    predicted_lap_times = []

    # Loop through each driver and predict lap time at the given circuit
    for driver_id in driver_ids:
        predicted_time = predict_lap_time(driver_id, circuit_id, average_lap_time)
        predicted_lap_times.append(predicted_time)

    # Calculate the average lap time for the race
    average_lap_time = np.mean(predicted_lap_times)

    print(f"Predicted Average Lap Time for Circuit {circuit_id}: {average_lap_time:.2f} seconds")

# Example usage
circuitId_bahrain = 3  # Example circuit ID for Bahrain
average_lap_time_placeholder = merged_data['lapTime'].mean()  # Use the average lap time as a placeholder

predict_average_race_lap_times(circuitId_bahrain, average_lap_time_placeholder)


NameError: name 'merged_data' is not defined